<oi>
    미국 상장된 회사의 경우 russel 3000 index의 경우 97%가 설명을 할 수 있다.<br>
    그렇기에 블룸버그 상 russel 3000 index 는 RAY index 이기에, 이 안의 index 멤버를 기반으로 회사들의 티커를 추출하고, <br>
    블룸버그 상에서는 미국 상장 회사의 경우 US Equity 가 티커 뒤에 붙어야 하므로 <br>
    블룸버그 티커들을 생성한 이후, 뒤에 US Equity를 붙여줌 <br>
    허나, 블룸버그 상에서 2500개 까지만 추출이 되는 탓에 이후 500개에 대해서는 손수 추출을 직접 한 후, 전처리를 진행함 <br>

## 러셀지수에서 기업명 다 긁어오기

In [8]:
from xbbg import blp
import pandas as pd

mem=blp.bds('RAY Index','INDX_MEMBERS')
#blp.bdp('RAY Index', 'NAME')
print(mem.head())

mem.rename(columns = {'member_ticker_and_exchange_code' : 'ticker'}, inplace=True) #컬럼명을 티커라는 명으로 편하게 바꾸어서 사용할때
mem.ticker=mem.ticker.str.split(' ').str[0] + ' US Equity'
print(mem.head())
mem.to_csv('RAY_mem.csv') #2500개 까지만 블룸버그 상에서 추출이 되므로 이후 500개에 대해서는 직접 복사 붙여넣기를 함

          member_ticker_and_exchange_code
RAY Index                            A UN
RAY Index                           AA UN
RAY Index                         AADI UR
RAY Index                          AAL UW
RAY Index                          AAN UN
                   ticker
RAY Index     A US Equity
RAY Index    AA US Equity
RAY Index  AADI US Equity
RAY Index   AAL US Equity
RAY Index   AAN US Equity


In [44]:
mem = pd.read_csv('RAY_mem.csv', encoding='cp949')
mem.ticker=mem.ticker.str.split(' ').str[0] + ' US Equity' 
#이전 언급과 마찬가지로, 2500까지만 추출이 되어서 이후 500개도 마찬가지 정제를 위하여 진행함
mem.to_csv('RAY_최종.csv', encoding='cp949')

tickers = mem['ticker'].to_list()
var_list = ['ESG_SCORE', 'E_SCORE', 'ESG_DISCLOSURE_SCORE', 'ENVIRON_DISCLOSURE_SCORE', 'ARDR_RAW_MATERIAL', 'ENERGY_CONSUMPTION', 'COAL_USED', 'OIL_DIESEL_USED', 'NAT_GAS_USED']
for var in var_list:
    #blp.bdh 는 historical data를 주기 때문에 Per='Y'를 통하여 회계연도별 데이터를 공시 날짜와 함께 받아와줌
    data = blp.bdh(tickers=tickers, flds = [var], start_date = '2005-01-01', end_date='2023-12-31', Fill = 'P', Per = 'Y')
    data.transpose().to_csv(var+'.csv', encoding='cp949')

In [9]:
#예시 결과 데이터
import pandas as pd
data = pd.read_csv('ESG_DISCLOSURE_SCORE.csv')
data[1:5].dropna(axis=1, how='all') #데이터가 존재하는 것만 예시로 확인해봄

,Unnamed: 0,Unnamed: 1,2005-12-31,2006-12-31,2007-12-29,2007-12-31,2008-12-31,2009-01-03,2009-12-31,2010-01-02,...,2017-12-31,2018-12-29,2018-12-31,2019-12-28,2019-12-31,2020-12-31,2021-01-02,2021-12-31,2022-01-01,2022-12-31
1,AA US Equity,ESG_DISCLOSURE_SCORE,NaN,41.0220,NaN,48.3754,39.8551,NaN,52.1074,NaN,...,68.9669,NaN,72.0652,NaN,74.1575,77.6984,NaN,80.4647,NaN,NaN
2,AAL US Equity,ESG_DISCLOSURE_SCORE,33.4775,35.1977,NaN,40.6700,40.6800,NaN,38.4971,NaN,...,55.5679,NaN,62.3579,NaN,62.8307,63.3739,NaN,63.3739,NaN,64.4201
3,AAON US Equity,ESG_DISCLOSURE_SCORE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.6568,NaN,31.9586,NaN,36.3746,38.5776,NaN,42.0380,NaN,40.4889
4,AAP US Equity,ESG_DISCLOSURE_SCORE,NaN,NaN,29.5443,NaN,NaN,30.4698,NaN,30.5804,...,NaN,36.0628,NaN,38.6078,NaN,NaN,47.7014,NaN,51.7051,51.7051


### 연(회계연도) 별 데이터로 환산하여 후에 wrds 등과 결합하기 위하여 진행한 결과

In [ ]:
import pandas as pd

csvname = ['ESG_DISCLOSURE_SCORE', 'ENVIRON_DISCLOSURE_SCORE', 'ARDR_RAW_MATERIAL', 'ENERGY_CONSUMPTION', 'COAL_USED', 'OIL_DIESEL_USED', 'NAT_GAS_USED', 'NUMBER_EMPLOYEES_CSR']

for name in csvname:
    df1 = pd.read_csv(name+'.csv', index_col=0).transpose()
    df1 = df1.drop(df1.index[0], axis=0)
    df1.index = pd.to_datetime(df1.index)
    df1['year'] = df1.index.map(lambda x: x.year if x.month >= 6 else x.year - 1)
    #df1['year'] = df1.index.year
    df2 = df1.groupby('year').mean()
    df2.to_csv(name+'_year.csv', encoding='cp949')

In [5]:
df2 = pd.read_csv('ESG_DISCLOSURE_SCORE_year.csv', encoding='cp949')
df2.head() #연별로  치환한 예시 데이터

,year,A US Equity,AA US Equity,AAL US Equity,AAON US Equity,AAP US Equity,AAPL US Equity,AAT US Equity,ABBV US Equity,ABCB US Equity,...,ZM US Equity,ZNTL US Equity,ZS US Equity,ZTS US Equity,ZUMZ US Equity,ZUO US Equity,ZVIA US Equity,ZWS US Equity,ZYME US Equity,ZYXI US Equity
0,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005,NaN,NaN,33.4775,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,28.8402,NaN,NaN,NaN,NaN,NaN
2,2006,46.1624,41.0220,35.1977,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,31.9787,NaN,NaN,NaN,NaN,NaN
3,2007,48.5565,48.3754,40.6700,NaN,29.5443,35.4693,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,31.9787,NaN,NaN,NaN,NaN,NaN
4,2008,48.3855,39.8551,40.6800,NaN,30.4698,42.3297,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,31.9787,NaN,NaN,NaN,NaN,NaN
